In [1]:
from tensorflow.keras import layers
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv("data/flipkart_com-ecommerce_sample_1050.csv")

In [35]:
df['categorie'] = [c[2:-1].split('>>')[0][0:-1].lower().replace(" ","_") for c in df["product_category_tree"]]
df = df[['categorie','description','image']]

# model nlp

In [38]:
df.head()

,categorie,description,image
0,home_furnishing,Key Features of Elegance Polyester Multicolor ...,55b85ea15a1536d46b7190ad6fff8ce7.jpg
1,baby_care,Specifications of Sathiyas Cotton Bath Towel (...,7b72c92c2f6c40268628ec5f14c6d590.jpg
2,baby_care,Key Features of Eurospa Cotton Terry Face Towe...,64d5d4a258243731dc7bbb1eef49ad74.jpg
3,home_furnishing,Key Features of SANTOSH ROYAL FASHION Cotton P...,d4684dcdc759dd9cdf41504698d737d8.jpg
4,home_furnishing,Key Features of Jaipur Print Cotton Floral Kin...,6325b6870c54cd47be6ebfbffa620ec7.jpg


In [40]:
nlp_in = layers.Input(shape=(len_voc,),name="nlp_in")
nlp_1 = layers.Dense(64,activation='relu',name="nlp_1")(nlp_in)


# model cv

In [41]:
cv_in = layers.Input(shape=(128,128,3),name="cv_in")
cv_1 = layers.Conv2D(32, (3, 3), activation='relu',input_shape=(128, 128, 3),name="cv_1")(cv_in)
cv_2 = layers.MaxPooling2D((2,2),name="cv_2")(cv_1)
cv_3 = layers.Conv2D(64, (3, 3), activation='relu',name="cv_3")(cv_2)
cv_4 = layers.MaxPooling2D((2,2),name="cv_4")(cv_3)
cv_5 = layers.Conv2D(64, (3, 3), activation='relu',name="cv_5")(cv_4)
cv_6 = layers.Flatten(name="cv_6")(cv_5)
cv_7 = layers.Dropout(0.2,name="cv_7")(cv_6)
cv_8 = layers.Dense(64,activation='relu',name="cv_8")(cv_7)

# fusion

In [42]:
#Merging model nlp and cv
fusion = layers.concatenate([nlp_1,cv_8],name="fusion_layer")
#Final Layer
output_layer = layers.Dense(7, activation = "softmax", name = "output_layer")(fusion)

In [36]:
import cv2
X_cv= []
for i in df["image"]:
    im = image = cv2.imread("data/Images_reshape/"+i)
    X_cv.append(im)
X_cv = np.array(X_cv)

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#df["image_file"] = X_cv
X = df['description']
y=df['categorie']
X_2, X_test, y_2, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_2, y_2, test_size=0.15, random_state=42)

X_2, X_test_cv, y_2, y_test = train_test_split(X_cv, y, test_size=0.15, random_state=42)
X_train_cv, X_val_cv, y_train, y_val = train_test_split(X_2, y_2, test_size=0.15, random_state=42)

vectorizer = CountVectorizer()
vectorizer.fit(X)
X_train_vect = vectorizer.transform(X_train)
X_test_vect = vectorizer.transform(X_test)
X_val_vect = vectorizer.transform(X_val)
len_voc = len(vectorizer.vocabulary_)

# je dois trasnformer ma variable cible en dummies 
# afin que chaque neurone de la dernière couche ait sa propre colone.
from tensorflow.keras.utils import to_categorical
#y_train.replace(["happy","sadness","anger","fear","love","surprise"],[0,1,2,3,4,5],inplace=True)
#y_test.replace(["happy","sadness","anger","fear","love","surprise"],[0,1,2,3,4,5],inplace=True)
y_train.replace(["beauty_and_personal_care","kitchen_&_dining","baby_care","home_decor_&_festive_needs","watches","home_furnishing","computers"],[0,1,2,3,4,5,6],inplace=True)
y_test.replace(["beauty_and_personal_care","kitchen_&_dining","baby_care","home_decor_&_festive_needs","watches","home_furnishing","computers"],[0,1,2,3,4,5,6],inplace=True)
y_val.replace(["beauty_and_personal_care","kitchen_&_dining","baby_care","home_decor_&_festive_needs","watches","home_furnishing","computers"],[0,1,2,3,4,5,6],inplace=True)

dummy_y_train = to_categorical( y_train)
dummy_y_test = to_categorical( y_test)
dummy_y_val = to_categorical( y_val)

In [43]:
from tensorflow.keras.models import Model
#Model Definition 
merged = Model(inputs=[(nlp_in,cv_in)],outputs=[output_layer], name = "merged_model")
#Model Details
merged.summary()

Model: "merged_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cv_in (InputLayer)              [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
cv_1 (Conv2D)                   (None, 126, 126, 32) 896         cv_in[0][0]                      
__________________________________________________________________________________________________
cv_2 (MaxPooling2D)             (None, 63, 63, 32)   0           cv_1[0][0]                       
__________________________________________________________________________________________________
cv_3 (Conv2D)                   (None, 61, 61, 64)   18496       cv_2[0][0]                       
_______________________________________________________________________________________

In [44]:
merged.compile(optimizer='adam', 
             loss="CategoricalCrossentropy",
             metrics=['accuracy'])

history = merged.fit([X_train_vect, X_train_cv], dummy_y_train, epochs=10, 
                    validation_data=([X_val_vect, X_val_cv], dummy_y_val),batch_size=32)

Epoch 1/10
24/24 [==============================] - 9s 361ms/step - loss: 44.5182 - accuracy: 0.3417 - val_loss: 1.4057 - val_accuracy: 0.6045
Epoch 2/10
24/24 [==============================] - 8s 346ms/step - loss: 0.8665 - accuracy: 0.8298 - val_loss: 0.9071 - val_accuracy: 0.8209
Epoch 3/10
24/24 [==============================] - 8s 346ms/step - loss: 0.3758 - accuracy: 0.9551 - val_loss: 0.7026 - val_accuracy: 0.8134
Epoch 4/10
24/24 [==============================] - 8s 346ms/step - loss: 0.2520 - accuracy: 0.9670 - val_loss: 0.7710 - val_accuracy: 0.7910
Epoch 5/10
24/24 [==============================] - 10s 409ms/step - loss: 0.1609 - accuracy: 0.9776 - val_loss: 0.6189 - val_accuracy: 0.8582
Epoch 6/10
24/24 [==============================] - 10s 402ms/step - loss: 0.1065 - accuracy: 0.9894 - val_loss: 0.8154 - val_accuracy: 0.8134
Epoch 7/10
24/24 [==============================] - 8s 346ms/step - loss: 0.0560 - accuracy: 0.9934 - val_loss: 0.7923 - val_accuracy: 0.7836
Epo

In [45]:
merged.evaluate([X_test_vect,X_test_cv],dummy_y_test)

5/5 [==============================] - 0s 66ms/step - loss: 1.4065 - accuracy: 0.7468


[1.4065269231796265, 0.746835470199585]